# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['HDFCBANK', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[2]
STOCK_SYMBOL

'JUBLFOOD'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df = stock_df.drop(columns = ['series', 'PREV. CLOSE'])
stock_df.columns = [
    'Date',
    'Open', 'High', 'Low', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
]
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2020-01-01,1656.95,1673.00,1652.80,1664.6,1663.15,1664.37,1673.00,1077.9,1142694,1.901871e+09,27428
1,2020-01-02,1665.00,1688.35,1655.50,1686.0,1682.70,1675.06,1688.35,1077.9,840514,1.407907e+09,26148
2,2020-01-03,1681.00,1696.90,1668.05,1679.0,1682.75,1685.46,1696.90,1077.9,975751,1.644585e+09,26219
3,2020-01-06,1670.30,1688.85,1664.25,1679.0,1679.60,1676.81,1696.90,1077.9,586899,9.841203e+08,20012
4,2020-01-07,1688.00,1702.50,1684.05,1689.4,1693.65,1693.57,1702.50,1077.9,845385,1.431717e+09,23848
...,...,...,...,...,...,...,...,...,...,...,...,...
1027,2024-01-29,508.75,518.45,504.10,518.0,513.15,510.72,586.95,412.1,6212204,3.172690e+09,77868
1028,2024-01-30,517.15,521.90,508.05,513.0,510.15,513.73,586.95,412.1,2464481,1.266071e+09,34342
1029,2024-01-31,514.00,526.15,509.50,518.2,519.55,518.54,586.95,412.1,4259420,2.208667e+09,92424
1030,2024-02-01,513.00,518.45,498.10,501.4,501.90,505.93,586.95,412.1,6644517,3.361652e+09,131585


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1032 non-null   datetime64[ns]
 1   Open     1032 non-null   float64       
 2   High     1032 non-null   float64       
 3   Low      1032 non-null   float64       
 4   LTP      1032 non-null   float64       
 5   Close    1032 non-null   float64       
 6   VWAP     1032 non-null   float64       
 7   52W H    1032 non-null   float64       
 8   52W L    1032 non-null   float64       
 9   Volume   1032 non-null   int64         
 10  Value    1032 non-null   float64       
 11  #Trades  1032 non-null   int64         
dtypes: datetime64[ns](1), float64(9), int64(2)
memory usage: 96.9 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Open']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh
824,2023-04-03,444.00,444.00,432.20,436.15,435.70,435.94,2861.00,412.10,1694155,7.385511e+08,41432,11.80,0,0,0
343,2021-04-28,2947.55,2966.95,2890.00,2913.05,2901.80,2933.85,3215.50,1466.55,390757,1.146423e+09,24998,76.95,0,0,0
1017,2024-01-12,531.40,531.80,523.40,526.50,525.65,526.31,586.95,412.10,1151034,6.058014e+08,33788,8.40,0,0,0
998,2023-12-15,577.95,577.95,562.55,565.00,565.45,567.73,577.95,412.10,1668246,9.471103e+08,37129,15.40,0,0,1
146,2020-07-14,1628.65,1665.55,1618.65,1655.00,1652.30,1645.01,1973.85,1077.90,943682,1.552366e+09,45329,46.90,1,0,0
358,2021-05-20,3009.00,3062.45,2967.40,2996.95,2985.80,3008.62,3215.50,1485.90,572691,1.723009e+09,35177,95.05,0,0,0
201,2020-09-29,2442.45,2446.25,2367.25,2375.95,2371.95,2395.44,2457.80,1138.25,746299,1.787715e+09,29526,79.00,0,0,0
409,2021-08-02,3770.00,3774.80,3725.00,3750.95,3733.95,3746.81,3816.40,1720.10,280318,1.050299e+09,20351,49.80,0,0,0
101,2020-05-11,1577.00,1606.45,1565.00,1574.80,1569.45,1582.61,1973.85,1077.90,827588,1.309753e+09,44398,41.45,0,0,0
555,2022-03-03,2897.00,2905.25,2790.00,2796.00,2799.45,2824.81,4590.00,2660.70,717424,2.026586e+09,74694,115.25,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      47.58
Is52WLow      1.65
Is52WHigh     6.88
dtype: float64

In [9]:
stock_df.shape

(1032, 16)

### Date based columns

In [10]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
907,2023-08-03,488.0,493.30,485.00,491.00,491.25,489.38,652.35,412.10,1147745,5.616805e+08,20166,8.30,1,0,0,3,8,2023,3,215,3,1
31,2020-02-12,1834.9,1859.90,1804.20,1820.00,1820.00,1834.78,1973.85,1077.90,1281489,2.351252e+09,33264,55.70,0,0,0,12,2,2020,2,43,1,1
54,2020-03-03,1670.0,1720.80,1626.05,1699.85,1696.45,1663.14,1973.85,1077.90,2594089,4.314341e+09,107137,94.75,1,0,0,3,3,2020,1,63,1,0
988,2023-12-01,563.2,567.00,559.70,565.00,565.40,563.97,567.40,412.10,1946123,1.097546e+09,39298,7.30,1,0,0,1,12,2023,4,335,4,1
931,2023-09-07,534.0,541.30,531.30,533.00,532.95,535.68,652.35,412.10,2639005,1.413651e+09,43676,10.00,0,0,0,7,9,2023,3,250,3,1
627,2022-06-17,509.7,509.70,487.00,491.65,490.55,495.92,4590.00,451.20,5731010,2.842149e+09,98805,22.70,0,0,0,17,6,2022,4,168,2,1
593,2022-04-29,546.0,562.00,543.55,544.80,546.00,553.76,4590.00,525.00,3314953,1.835693e+09,80182,18.45,1,0,0,29,4,2022,4,119,2,1
378,2021-06-17,3220.0,3289.50,3193.25,3254.00,3253.60,3254.03,3332.70,1596.05,1095758,3.565630e+09,58937,96.25,1,0,0,17,6,2021,3,168,2,1
354,2021-05-14,2839.0,2856.90,2787.10,2814.35,2805.25,2811.14,3215.50,1485.90,309144,8.690460e+08,21769,69.80,0,0,0,14,5,2021,4,134,2,2
488,2021-11-26,3730.0,3742.45,3602.00,3650.00,3658.50,3660.44,4590.00,2392.30,441537,1.616220e+09,43332,140.45,0,0,0,26,11,2021,4,330,4,1


In [11]:
stock_df.shape

(1032, 23)

### Moving averages

#### `Close`

In [12]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,1663.15,1663.15,1663.15,1663.15,1663.15
1,1682.70,1672.93,1672.93,1672.93,1672.93
2,1682.75,1676.20,1676.20,1676.20,1676.20
3,1679.60,1677.05,1677.05,1677.05,1677.05
4,1693.65,1680.37,1680.37,1680.37,1680.37
...,...,...,...,...,...
1027,513.15,520.95,524.86,544.52,539.78
1028,510.15,518.93,523.25,542.67,539.93
1029,519.55,517.20,522.91,541.35,540.14
1030,501.90,514.16,521.51,539.25,540.04


#### `Range`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,20.20,20.20,20.20,20.20,20.20
1,32.85,26.52,26.52,26.52,26.52
2,28.85,27.30,27.30,27.30,27.30
3,24.60,26.62,26.62,26.62,26.62
4,18.45,24.99,24.99,24.99,24.99
...,...,...,...,...,...
1027,14.35,14.04,13.58,13.55,11.82
1028,13.85,14.17,12.90,13.50,11.96
1029,16.65,15.08,12.91,13.80,12.12
1030,20.35,15.99,13.71,13.99,12.34


#### `VWAP`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,1664.37,1664.37,1664.37,1664.37,1664.37
1,1675.06,1669.72,1669.72,1669.72,1669.72
2,1685.46,1674.96,1674.96,1674.96,1674.96
3,1676.81,1675.42,1675.42,1675.42,1675.42
4,1693.57,1679.05,1679.05,1679.05,1679.05
...,...,...,...,...,...
1027,510.72,520.32,525.42,545.46,540.22
1028,513.73,519.10,523.67,543.66,540.42
1029,518.54,517.10,522.96,542.20,540.61
1030,505.93,514.12,521.94,540.13,540.57


#### `Volume`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1142694,1142694,1142694,1142694,1142694
1,840514,991604,991604,991604,991604
2,975751,986319,986319,986319,986319
3,586899,886464,886464,886464,886464
4,845385,878248,878248,878248,878248
...,...,...,...,...,...
1027,6212204,3169424,2587621,2427123,2134424
1028,2464481,3160963,2591987,2453664,2166395
1029,4259420,3445825,2697419,2544398,2211641
1030,6644517,4301178,3028044,2686249,2305913


#### `Value`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.901871e+09,1901871196,1901871196,1901871196,1901871196
1,1.407907e+09,1654889284,1654889284,1654889284,1654889284
2,1.644585e+09,1651454555,1651454555,1651454555,1651454555
3,9.841203e+08,1484620980,1484620980,1484620980,1484620980
4,1.431717e+09,1474040255,1474040255,1474040255,1474040255
...,...,...,...,...,...
1027,3.172690e+09,1640447875,1353563596,1320179854,1158761061
1028,1.266071e+09,1633013466,1351599355,1330811888,1175295089
1029,2.208667e+09,1776201812,1404377593,1375624151,1199049589
1030,3.361652e+09,2206989109,1569930520,1442436250,1246708400


#### `#Trades`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,27428,27428,27428,27428,27428
1,26148,26788,26788,26788,26788
2,26219,26598,26598,26598,26598
3,20012,24951,24951,24951,24951
4,23848,24731,24731,24731,24731
...,...,...,...,...,...
1027,77868,64065,56940,50662,42078
1028,34342,58966,54449,50569,42443
1029,92424,60480,58094,52034,43717
1030,131585,76467,64211,54870,45393


### Target columns

In [18]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1679.60,1727.80,1773.80,1825.55
1,1693.65,1731.35,1793.55,1820.00
2,1725.70,1720.50,1810.10,1835.10
3,1714.30,1753.45,1822.90,1812.60
4,1727.80,1746.70,1754.30,1833.05
...,...,...,...,...
1027,501.90,NaN,NaN,NaN
1028,494.85,NaN,NaN,NaN
1029,NaN,NaN,NaN,NaN
1030,NaN,NaN,NaN,NaN


### Saving processed data

In [19]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1656.95,1673.00,1652.80,1664.6,1663.15,1664.37,1673.00,1077.9,1142694,1.901871e+09,27428,20.20,1,0,1,1,1,2020,2,1,1,1,1663.15,1663.15,1663.15,1663.15,20.20,20.20,20.20,20.20,1664.37,1664.37,1664.37,1664.37,1142694,1142694,1142694,1142694,1901871196,1901871196,1901871196,1901871196,27428,27428,27428,27428,1679.60,1727.80,1773.80,1825.55
1,2020-01-02,1665.00,1688.35,1655.50,1686.0,1682.70,1675.06,1688.35,1077.9,840514,1.407907e+09,26148,32.85,1,0,1,2,1,2020,3,2,1,1,1672.93,1672.93,1672.93,1672.93,26.52,26.52,26.52,26.52,1669.72,1669.72,1669.72,1669.72,991604,991604,991604,991604,1654889284,1654889284,1654889284,1654889284,26788,26788,26788,26788,1693.65,1731.35,1793.55,1820.00
2,2020-01-03,1681.00,1696.90,1668.05,1679.0,1682.75,1685.46,1696.90,1077.9,975751,1.644585e+09,26219,28.85,1,0,1,3,1,2020,4,3,1,1,1676.20,1676.20,1676.20,1676.20,27.30,27.30,27.30,27.30,1674.96,1674.96,1674.96,1674.96,986319,986319,986319,986319,1651454555,1651454555,1651454555,1651454555,26598,26598,26598,26598,1725.70,1720.50,1810.10,1835.10
3,2020-01-06,1670.30,1688.85,1664.25,1679.0,1679.60,1676.81,1696.90,1077.9,586899,9.841203e+08,20012,24.60,1,0,0,6,1,2020,0,6,1,3,1677.05,1677.05,1677.05,1677.05,26.62,26.62,26.62,26.62,1675.42,1675.42,1675.42,1675.42,886464,886464,886464,886464,1484620980,1484620980,1484620980,1484620980,24951,24951,24951,24951,1714.30,1753.45,1822.90,1812.60
4,2020-01-07,1688.00,1702.50,1684.05,1689.4,1693.65,1693.57,1702.50,1077.9,845385,1.431717e+09,23848,18.45,1,0,1,7,1,2020,1,7,1,1,1680.37,1680.37,1680.37,1680.37,24.99,24.99,24.99,24.99,1679.05,1679.05,1679.05,1679.05,878248,878248,878248,878248,1474040255,1474040255,1474040255,1474040255,24731,24731,24731,24731,1727.80,1746.70,1754.30,1833.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,2024-01-29,508.75,518.45,504.10,518.0,513.15,510.72,586.95,412.1,6212204,3.172690e+09,77868,14.35,1,0,0,29,1,2024,0,29,1,4,520.95,524.86,544.52,539.78,14.04,13.58,13.55,11.82,520.32,525.42,545.46,540.22,3169424,2587621,2427123,2134424,1640447875,1353563596,1320179854,1158761061,64065,56940,50662,42078,501.90,NaN,NaN,NaN
1028,2024-01-30,517.15,521.90,508.05,513.0,510.15,513.73,586.95,412.1,2464481,1.266071e+09,34342,13.85,0,0,0,30,1,2024,1,30,1,1,518.93,523.25,542.67,539.93,14.17,12.90,13.50,11.96,519.10,523.67,543.66,540.42,3160963,2591987,2453664,2166395,1633013466,1351599355,1330811888,1175295089,58966,54449,50569,42443,494.85,NaN,NaN,NaN
1029,2024-01-31,514.00,526.15,509.50,518.2,519.55,518.54,586.95,412.1,4259420,2.208667e+09,92424,16.65,1,0,0,31,1,2024,2,31,1,1,517.20,522.91,541.35,540.14,15.08,12.91,13.80,12.12,517.10,522.96,542.20,540.61,3445825,2697419,2544398,2211641,1776201812,1404377593,1375624151,1199049589,60480,58094,52034,43717,NaN,NaN,NaN,NaN
1030,2024-02-01,513.00,518.45,498.10,501.4,501.90,505.93,586.95,412.1,6644517,3.361652e+09,131585,20.35,0,0,0,1,2,2024,3,32,1,1,514.16,521.51,539.25,540.04,15.99,13.71,13.99,12.34,514.12,521.94,540.13,540.57,4301178,3028044,2686249,2305913,2206989109,1569930520,1442436250,1246708400,76467,64211,54870,45393,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [20]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [21]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.996,1.006,0.994,1.001,1.006,0.648,1.0,1.000,1.000,1.000,1.000,0.012,0.012,0.012,0.012,0.012,1.001,1.001,1.001,1.001,1.001,1.010,1.039,1.067,1.098
1,0.989,1.003,0.984,1.002,1.003,0.641,1.0,0.994,0.994,0.994,0.994,0.020,0.016,0.016,0.016,0.016,0.995,0.992,0.992,0.992,0.992,1.007,1.029,1.066,1.082
2,0.999,1.008,0.991,0.998,1.008,0.641,1.0,0.996,0.996,0.996,0.996,0.017,0.016,0.016,0.016,0.016,1.002,0.995,0.995,0.995,0.995,1.026,1.022,1.076,1.091
3,0.994,1.006,0.991,1.000,1.010,0.642,1.0,0.998,0.998,0.998,0.998,0.015,0.016,0.016,0.016,0.016,0.998,0.998,0.998,0.998,0.998,1.021,1.044,1.085,1.079
4,0.997,1.005,0.994,0.997,1.005,0.636,1.0,0.992,0.992,0.992,0.992,0.011,0.015,0.015,0.015,0.015,1.000,0.991,0.991,0.991,0.991,1.020,1.031,1.036,1.082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,0.991,1.010,0.982,1.009,1.144,0.803,1.0,1.015,1.023,1.061,1.052,0.028,0.027,0.026,0.026,0.023,0.995,1.014,1.024,1.063,1.053,0.978,NaN,NaN,NaN
1028,1.014,1.023,0.996,1.006,1.151,0.808,1.0,1.017,1.026,1.064,1.058,0.027,0.028,0.025,0.026,0.023,1.007,1.018,1.027,1.066,1.059,0.970,NaN,NaN,NaN
1029,0.989,1.013,0.981,0.997,1.130,0.793,1.0,0.995,1.006,1.042,1.040,0.032,0.029,0.025,0.027,0.023,0.998,0.995,1.007,1.044,1.041,NaN,NaN,NaN,NaN
1030,1.022,1.033,0.992,0.999,1.169,0.821,1.0,1.024,1.039,1.074,1.076,0.041,0.032,0.027,0.028,0.025,1.008,1.024,1.040,1.076,1.077,NaN,NaN,NaN,NaN


### Volume based columns

In [22]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [23]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,1.180,1.180,1.180,1.180
2,1.0,1.011,1.011,1.011,1.011
3,1.0,1.510,1.510,1.510,1.510
4,1.0,1.039,1.039,1.039,1.039
...,...,...,...,...,...
1027,1.0,0.510,0.417,0.391,0.344
1028,1.0,1.283,1.052,0.996,0.879
1029,1.0,0.809,0.633,0.597,0.519
1030,1.0,0.647,0.456,0.404,0.347


### Value based columns

In [24]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [25]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,1.175,1.175,1.175,1.175
2,1.0,1.004,1.004,1.004,1.004
3,1.0,1.509,1.509,1.509,1.509
4,1.0,1.030,1.030,1.030,1.030
...,...,...,...,...,...
1027,1.0,0.517,0.427,0.416,0.365
1028,1.0,1.290,1.068,1.051,0.928
1029,1.0,0.804,0.636,0.623,0.543
1030,1.0,0.657,0.467,0.429,0.371


### Trade count based columns

In [26]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [27]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,1.024,1.024,1.024,1.024
2,1.0,1.014,1.014,1.014,1.014
3,1.0,1.247,1.247,1.247,1.247
4,1.0,1.037,1.037,1.037,1.037
...,...,...,...,...,...
1027,1.0,0.823,0.731,0.651,0.540
1028,1.0,1.717,1.585,1.473,1.236
1029,1.0,0.654,0.629,0.563,0.473
1030,1.0,0.581,0.488,0.417,0.345


### Dropping unnecessary columns

In [28]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.996,1.006,0.994,1.001,1.001,1.006,0.648,1,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.012,0.012,0.012,0.012,1.001,1.001,1.001,1.001,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.010,1.039,1.067,1.098
1,0.989,1.003,0.984,1.002,0.995,1.003,0.641,1,0,1,2,1,2020,3,2,1,1,0.994,0.994,0.994,0.994,0.016,0.016,0.016,0.016,0.992,0.992,0.992,0.992,1.180,1.180,1.180,1.180,1.175,1.175,1.175,1.175,1.024,1.024,1.024,1.024,1.007,1.029,1.066,1.082
2,0.999,1.008,0.991,0.998,1.002,1.008,0.641,1,0,1,3,1,2020,4,3,1,1,0.996,0.996,0.996,0.996,0.016,0.016,0.016,0.016,0.995,0.995,0.995,0.995,1.011,1.011,1.011,1.011,1.004,1.004,1.004,1.004,1.014,1.014,1.014,1.014,1.026,1.022,1.076,1.091
3,0.994,1.006,0.991,1.000,0.998,1.010,0.642,1,0,0,6,1,2020,0,6,1,3,0.998,0.998,0.998,0.998,0.016,0.016,0.016,0.016,0.998,0.998,0.998,0.998,1.510,1.510,1.510,1.510,1.509,1.509,1.509,1.509,1.247,1.247,1.247,1.247,1.021,1.044,1.085,1.079
4,0.997,1.005,0.994,0.997,1.000,1.005,0.636,1,0,1,7,1,2020,1,7,1,1,0.992,0.992,0.992,0.992,0.015,0.015,0.015,0.015,0.991,0.991,0.991,0.991,1.039,1.039,1.039,1.039,1.030,1.030,1.030,1.030,1.037,1.037,1.037,1.037,1.020,1.031,1.036,1.082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,0.991,1.010,0.982,1.009,0.995,1.144,0.803,1,0,0,29,1,2024,0,29,1,4,1.015,1.023,1.061,1.052,0.027,0.026,0.026,0.023,1.014,1.024,1.063,1.053,0.510,0.417,0.391,0.344,0.517,0.427,0.416,0.365,0.823,0.731,0.651,0.540,0.978,NaN,NaN,NaN
1028,1.014,1.023,0.996,1.006,1.007,1.151,0.808,0,0,0,30,1,2024,1,30,1,1,1.017,1.026,1.064,1.058,0.028,0.025,0.026,0.023,1.018,1.027,1.066,1.059,1.283,1.052,0.996,0.879,1.290,1.068,1.051,0.928,1.717,1.585,1.473,1.236,0.970,NaN,NaN,NaN
1029,0.989,1.013,0.981,0.997,0.998,1.130,0.793,1,0,0,31,1,2024,2,31,1,1,0.995,1.006,1.042,1.040,0.029,0.025,0.027,0.023,0.995,1.007,1.044,1.041,0.809,0.633,0.597,0.519,0.804,0.636,0.623,0.543,0.654,0.629,0.563,0.473,NaN,NaN,NaN,NaN
1030,1.022,1.033,0.992,0.999,1.008,1.169,0.821,0,0,0,1,2,2024,3,32,1,1,1.024,1.039,1.074,1.076,0.032,0.027,0.028,0.025,1.024,1.040,1.076,1.077,0.647,0.456,0.404,0.347,0.657,0.467,0.429,0.371,0.581,0.488,0.417,0.345,NaN,NaN,NaN,NaN


### Saving standardized data

In [29]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.996,1.006,0.994,1.001,1.001,1.006,0.648,1,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.012,0.012,0.012,0.012,1.001,1.001,1.001,1.001,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.010,1.039,1.067,1.098
1,0.989,1.003,0.984,1.002,0.995,1.003,0.641,1,0,1,2,1,2020,3,2,1,1,0.994,0.994,0.994,0.994,0.016,0.016,0.016,0.016,0.992,0.992,0.992,0.992,1.180,1.180,1.180,1.180,1.175,1.175,1.175,1.175,1.024,1.024,1.024,1.024,1.007,1.029,1.066,1.082
2,0.999,1.008,0.991,0.998,1.002,1.008,0.641,1,0,1,3,1,2020,4,3,1,1,0.996,0.996,0.996,0.996,0.016,0.016,0.016,0.016,0.995,0.995,0.995,0.995,1.011,1.011,1.011,1.011,1.004,1.004,1.004,1.004,1.014,1.014,1.014,1.014,1.026,1.022,1.076,1.091
3,0.994,1.006,0.991,1.000,0.998,1.010,0.642,1,0,0,6,1,2020,0,6,1,3,0.998,0.998,0.998,0.998,0.016,0.016,0.016,0.016,0.998,0.998,0.998,0.998,1.510,1.510,1.510,1.510,1.509,1.509,1.509,1.509,1.247,1.247,1.247,1.247,1.021,1.044,1.085,1.079
4,0.997,1.005,0.994,0.997,1.000,1.005,0.636,1,0,1,7,1,2020,1,7,1,1,0.992,0.992,0.992,0.992,0.015,0.015,0.015,0.015,0.991,0.991,0.991,0.991,1.039,1.039,1.039,1.039,1.030,1.030,1.030,1.030,1.037,1.037,1.037,1.037,1.020,1.031,1.036,1.082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,0.991,1.010,0.982,1.009,0.995,1.144,0.803,1,0,0,29,1,2024,0,29,1,4,1.015,1.023,1.061,1.052,0.027,0.026,0.026,0.023,1.014,1.024,1.063,1.053,0.510,0.417,0.391,0.344,0.517,0.427,0.416,0.365,0.823,0.731,0.651,0.540,0.978,NaN,NaN,NaN
1028,1.014,1.023,0.996,1.006,1.007,1.151,0.808,0,0,0,30,1,2024,1,30,1,1,1.017,1.026,1.064,1.058,0.028,0.025,0.026,0.023,1.018,1.027,1.066,1.059,1.283,1.052,0.996,0.879,1.290,1.068,1.051,0.928,1.717,1.585,1.473,1.236,0.970,NaN,NaN,NaN
1029,0.989,1.013,0.981,0.997,0.998,1.130,0.793,1,0,0,31,1,2024,2,31,1,1,0.995,1.006,1.042,1.040,0.029,0.025,0.027,0.023,0.995,1.007,1.044,1.041,0.809,0.633,0.597,0.519,0.804,0.636,0.623,0.543,0.654,0.629,0.563,0.473,NaN,NaN,NaN,NaN
1030,1.022,1.033,0.992,0.999,1.008,1.169,0.821,0,0,0,1,2,2024,3,32,1,1,1.024,1.039,1.074,1.076,0.032,0.027,0.028,0.025,1.024,1.040,1.076,1.077,0.647,0.456,0.404,0.347,0.657,0.467,0.429,0.371,0.581,0.488,0.417,0.345,NaN,NaN,NaN,NaN
